### Inicialização

In [1]:
import pandas as pd
import xlrd
import openpyxl
import requests
import googlemaps
import time
import xlsxwriter
from datetime import datetime
from typing import Optional

In [2]:
gmaps = googlemaps.Client(key="Insert key")

In [3]:
df_unidades = pd.read_excel("../data/SampleAddress.xlsx", sheet_name="Dasa")
df_clientes = pd.read_excel("../data/SampleAddress.xlsx", sheet_name="Cliente")

### Visualização dos dados

In [4]:
df_unidades.head(1)

,Nome site,CÓDIGO SAP,TIPO DE UNIDADE,MARCA,PAÍS,ESTADO,REGIONAL,ENDEREÇO,CEP,CIDADE,RESPONSÁVEL
0,Jardins,E38,PROPRIA,Alta Excelência Diagnóstica,Brasil,SP,SP,"Avenida Brasil, 1802, Bairro Jardim América, C...",01431-001,SAO PAULO,Roberto Papaleo


In [5]:
df_clientes.head(1)

,BU,Mundo,x2,Planta,Country,Média de Funcionários,Medical,Endereço,Local,CEP
0,BR,Supply,BR34,F. AGUDOS,Brazil,600,4,Rodovia Marechal Rondon Km 31,Agudos,17120-000


### Limpeza dos dados

In [6]:
df_unidades = df_unidades.dropna(subset=["CEP"])
df_clientes = df_clientes.dropna(subset=["CEP"])

### Extrair coordenadas dos cep

In [7]:
# Função que usa o google maps para extrair coord
def cep2coord(cep: str) -> Optional[dict]:
    # Geocoding an address
    geocode_result = gmaps.geocode(cep + ", Brasil")

    time.sleep(1)
    
    if len(geocode_result) > 0:
        return geocode_result[0]["geometry"]["location"]
    
    return {
        "lat": None,
        "lng": None
    }

In [8]:
# Cria duas novas colunas em df_unidade
df_unidades["lat"] = None
df_unidades["lng"] = None

In [9]:
df_unidades.head(1)

,Nome site,CÓDIGO SAP,TIPO DE UNIDADE,MARCA,PAÍS,ESTADO,REGIONAL,ENDEREÇO,CEP,CIDADE,RESPONSÁVEL,lat,lng
0,Jardins,E38,PROPRIA,Alta Excelência Diagnóstica,Brasil,SP,SP,"Avenida Brasil, 1802, Bairro Jardim América, C...",01431-001,SAO PAULO,Roberto Papaleo,None,None


In [10]:
# Para cada row de de_unidades, pega a lat e lng com sleep de 1s para n sobrecarregar a API
for row in df_unidades.iterrows():
    cep = row[1]["CEP"]
    
    coord = cep2coord(cep)
    
    row[1]["lat"] = coord["lat"]
    row[1]["lng"] = coord["lng"]
    
    time.sleep(0.1)

In [11]:
df_unidades.head(1)

,Nome site,CÓDIGO SAP,TIPO DE UNIDADE,MARCA,PAÍS,ESTADO,REGIONAL,ENDEREÇO,CEP,CIDADE,RESPONSÁVEL,lat,lng
0,Jardins,E38,PROPRIA,Alta Excelência Diagnóstica,Brasil,SP,SP,"Avenida Brasil, 1802, Bairro Jardim América, C...",01431-001,SAO PAULO,Roberto Papaleo,-23.5678,-46.6737


In [12]:
# Cria duas novas colunas em df_clientes
df_clientes["lat"] = None
df_clientes["lng"] = None

In [13]:
# Para cada row de de_clientes, pega a lat e lng com sleep de 1s para n sobrecarregar a API
for index, row in df_clientes.iterrows():
    cep = row["CEP"]
    
    coord = cep2coord(cep)
    
    df_clientes.loc[index, "lat"] = coord["lat"]
    df_clientes.loc[index, "lng"] = coord["lng"]
    
    time.sleep(0.1)

In [14]:
df_clientes.head(1)

,BU,Mundo,x2,Planta,Country,Média de Funcionários,Medical,Endereço,Local,CEP,lat,lng
0,BR,Supply,BR34,F. AGUDOS,Brazil,600,4,Rodovia Marechal Rondon Km 31,Agudos,17120-000,-22.5699,-49.0817


### Calcula distância usando o openstreetmap

In [15]:
df_unidades = df_unidades.dropna(subset=["lat", "lng"])
df_clientes = df_clientes.dropna(subset=["lat", "lng"])

In [16]:
df_result = []


for index_cliente, row_cliente in df_clientes.iterrows():
    
    shortest_distance = None
    unidade_cep = None
    
    cliente_coord = str(row_cliente["lat"]) + ", " + str(row_cliente["lng"])

    for index_unidade, row_unidade in df_unidades.iterrows():
        unidade_coord = str(row_unidade["lat"]) + ", " + str(row_unidade["lng"])
        
        directions_result = gmaps.directions(unidade_coord, cliente_coord, mode="transit")
        
        if len(directions_result) > 0:
            distance = directions_result[0]["legs"][0]["distance"]["value"]
            if (shortest_distance is None and unidade_cep is None) or (distance < shortest_distance):
                shortest_distance = distance
                unidade_cep = row_unidade["CEP"]
            
        time.sleep(2)
        
    df_result.append({
        "cliente_cep": row_cliente["CEP"],
        "unidade_cep": unidade_cep,
        "distance": shortest_distance
    })
        
df_result = pd.DataFrame(df_result)

In [17]:
df_result.head(30)

,cliente_cep,unidade_cep,distance
0,17120-000,None,NaN
1,07158-900,02085-000,36183.0
2,17120-000,None,NaN
3,16018-805,None,NaN
4,14808-156,None,NaN
5,06397-110,06693-005,15058.0
6,09950-300,09750-670,4438.0
7,06815-490,04661-200,20395.0
8,07251-250,02085-000,28790.0
9,09372-050,09750-670,19482.0


In [ ]:
writer = pd.ExcelWriter('../data/SampleAddress_processed.xlsx', engine='xlsxwriter')

df_unidades.to_excel(writer, sheet_name='Unidades')
df_clientes.to_excel(writer, sheet_name='Clientes')
df_result.to_excel(writer, sheet_name='Distancias')

writer.save()